In [1]:
import os
import pprint
import math
import time
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../tfmodels')
import bottleneck as bn

import random
import numpy as np
from past.builtins import xrange
import pandas as pd

#from data import read_data
from memory_model import *

In [2]:
flags = tf.app.flags

flags.DEFINE_integer("nb_words", 100001, "term number in input sequence(zero mask)")
flags.DEFINE_integer("mem_size", 30, "the memory length of input sequence")
flags.DEFINE_integer("nhop", 6, "the number of hop layers")
flags.DEFINE_integer("embedding_size", 300, "word embedding size")
flags.DEFINE_float("init_hid", 0.1, "init_hid for q")
flags.DEFINE_float("init_std", 0.05, "init_std for training variable")

flags.DEFINE_float("linear_ratio", 0.5, "keep probability of drop out [0.9]")
flags.DEFINE_float("learning_rate", 0.01, "learning rate [0.001]")
flags.DEFINE_integer("batch_size", 1024, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 50.0, "clip gradients to this norm")
flags.DEFINE_integer("n_epochs", 100, "number of epoch to use during training")
flags.DEFINE_boolean("epoch_save", True, "save checkpoint or not in each epoch")
flags.DEFINE_integer("print_step", 500, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", True, "use lr annealing or not after each epoch [False]")
#flags.DEFINE_string("current_task_name", '_season_prediction', "current task name [self_prediction]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 4, "gpu_num")

FLAGS = flags.FLAGS

In [3]:
h5_file_path = 'Data/episode_seq_28_for_memory.h5'
h5f = h5py.File(h5_file_path, 'r')
previous_videos = np.array(h5f['previous_videos'])
#previous_viewthrus = h5f['previous_viewthrus']
next_seasons = np.array(h5f['next_seasons'])

In [5]:
idxs = np.arange(0, samples)
train_idxs, test_idxs = train_test_split(idxs, test_size=0.2, random_state=42)
test_idxs = np.sort(test_idxs)

In [3]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    mem_model = MemN2NModel(FLAGS, session, 'memn2n_autoplay_prediction')
    mem_model.build_model(type='self', accK=5, nb_class=15001, export_attention=True)
    mem_model.build_model_summary()
    display(mem_model.model_summary())
    #mem_model.run([previous_urls,next_urls], train_idxs, test_idxs)

Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1.0
1,gpu/memory/Ain_c/embedding_table:0,"[100001, 300]",30000300.0
2,gpu/memory/Ain_t/W:0,"[30, 300]",9000.0
3,gpu/memory/Cin_c/embedding_table:0,"[100001, 300]",30000300.0
4,gpu/memory/Cin_t/W:0,"[30, 300]",9000.0
5,gpu/momory_hops/hops_h/W:0,"[300, 300]",90000.0
6,gpu/prediction/fc/W:0,"[300, 100001]",30000300.0
